# Cyber Crime Science Data Analysis and Visualization Project

## Data Import

### Libraries

In [1]:
import pandas as pd
import numpy as np


In [2]:
slachtofferschap_source = "data/source/slachtofferschap.csv"
age_distribution_source = "data/source/age_distribution.csv"
digitale_vaardigheden_source = "data/source/digitale_vaardigheden.csv"
education_levels_source = "data/source/education_levels.csv"
gdp_source = "data/source/gdp.csv"
bevolkingdichtheid_source = "data/source/bevolkingdichtheid.csv"
vestigingen_source = "data/source/vestigingen.csv"

In [3]:
slachtofferschap = pd.read_csv(slachtofferschap_source)
age_distribution = pd.read_csv(age_distribution_source, sep=';')
digitale_vaardigheden = pd.read_csv(digitale_vaardigheden_source, sep=',')
education_levels = pd.read_csv(education_levels_source, sep=';')
gdp = pd.read_csv(gdp_source, sep=';')
bevolkingdichtheid = pd.read_csv(bevolkingdichtheid_source, sep=';')
vestigingen = pd.read_csv(vestigingen_source, sep=';')

## Data Cleaning

### Keep only the desired columns

In [4]:
for col in vestigingen.columns:
    if col != "Regio's":
        vestigingen[col] = pd.to_numeric(vestigingen[col].str.replace('"', ''), errors='coerce').astype('Int64')


bevolkingdichtheid_filtered = bevolkingdichtheid[[
    "Regio's",
    "Bevolking/Bevolkingssamenstelling op 1 januari/Bevolkingsdichtheid (aantal inwoners per km²)"
]]

bevolkingdichtheid_filtered = bevolkingdichtheid_filtered.rename(columns={
    "Bevolking/Bevolkingssamenstelling op 1 januari/Bevolkingsdichtheid (aantal inwoners per km²)": "Bevolkingsdichtheid (aantal inwoners per km²)"
})



gdp_filtered = gdp[[
    "Regio's",
    "Bbp (marktprijzen) (mln euro)",
    "Bbp per inwoner (euro)",
    "Bbp, volumemutaties (%)",
    "Bruto toegevoegde waarde, volumemutaties (%)",
    "Toegevoegde waarde (bruto, basisprijzen) (mln euro)",
]]



education_levels_filtered = education_levels[[
    "Regio's",
    "Onderwijsniveau 5 categorieën/11 Basisonderwijs (%)",
    "Onderwijsniveau 5 categorieën/12 Vmbo, havo-, vwo-onderbouw, mbo1 (%)",
    "Onderwijsniveau 5 categorieën/21 Havo, vwo, mbo2-4 (%)",
    "Onderwijsniveau 5 categorieën/31 Hbo-, wo-bachelor (%)",
    "Onderwijsniveau 5 categorieën/32 Hbo-, wo-master, doctor (%)",
]]

education_levels_filtered = education_levels_filtered.rename(columns={
    "Regio's": "Regio's",
    "Onderwijsniveau 5 categorieën/11 Basisonderwijs (%)": "Basisonderwijs (%)",
    "Onderwijsniveau 5 categorieën/12 Vmbo, havo-, vwo-onderbouw, mbo1 (%)": "Vmbo/Havo/Vwo/Mbo1 (%)",
    "Onderwijsniveau 5 categorieën/21 Havo, vwo, mbo2-4 (%)": "Havo/Vwo/Mbo2-4 (%)",
    "Onderwijsniveau 5 categorieën/31 Hbo-, wo-bachelor (%)": "Hbo/Wo-bachelor (%)",
    "Onderwijsniveau 5 categorieën/32 Hbo-, wo-master, doctor (%)": "Hbo/Wo-master/Doctor (%)"
})



age_distribution_filtered = age_distribution[[
    "Regio's",
    "Bevolking op 1 januari (aantal)",
    "Gemiddelde bevolking  (aantal)"
]]


slachtofferschap_filtered = slachtofferschap[[
    "Regio's",
    "Online criminaliteit Totaal slachtoffers (%)",
    "Phishing Totaal slachtoffers (%)"
]]

# Remove rows where the 'Regio's' column is 'Nederland'
slachtofferschap_filtered = slachtofferschap_filtered[slachtofferschap_filtered["Regio's"] != "Nederland"]


## Merge Datasets

In [5]:
merged_df = pd.merge(
    slachtofferschap_filtered,
    age_distribution_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

merged_df = pd.merge(
    merged_df,
    education_levels_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

merged_df = pd.merge(
    merged_df,
    education_levels_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

merged_df = pd.merge(
    merged_df,
    education_levels_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

merged_df = pd.merge(
    merged_df,
    bevolkingdichtheid_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

vestigingen["Regio's"] = vestigingen["Regio's"].str.strip()
merged_df["Regio's"] = merged_df["Regio's"].str.strip()

merged_df = pd.merge(
    merged_df,
    vestigingen,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)


display(merged_df)

,Regio's,Online criminaliteit Totaal slachtoffers (%),Phishing Totaal slachtoffers (%),Bevolking op 1 januari (aantal),Gemiddelde bevolking (aantal),Basisonderwijs (%)_x,Vmbo/Havo/Vwo/Mbo1 (%)_x,Havo/Vwo/Mbo2-4 (%)_x,Hbo/Wo-bachelor (%)_x,Hbo/Wo-master/Doctor (%)_x,...,"""L Verhuur en handel van onroerend goed""","""M Specialistische zakelijke diensten""","""N Verhuur en overige zakelijke diensten""","""O Openbaar bestuur en overheidsdiensten""","""P Onderwijs""","""Q Gezondheids- en welzijnszorg""","""R Cultuur, sport en recreatie""","""S Overige dienstverlening""","""T Huishoudens""","""U Extraterritoriale organisaties"""
0,Groningen (PV),"16,3","1,0",596075,598711.0,7.4,16.3,45.1,20.0,11.2,...,895,10515,2925,225,3875,7160,4510,3960,0,0
1,Fryslân (PV),"14,2","0,6",659551,660727.5,7.8,19.1,47.0,19.7,6.4,...,1080,12120,3450,190,4125,7985,4160,4485,0,0
2,Drenthe (PV),"14,8","1,0",502051,503090.0,8.4,18.9,46.5,18.7,7.6,...,740,8465,2430,120,3010,6525,2405,3640,0,0
3,Overijssel (PV),"14,8","0,6",1184333,1186683.0,8.2,18.1,44.9,20.3,8.5,...,1905,21150,5425,245,7160,15045,6465,7635,5,0
4,Flevoland (PV),"16,9","0,9",444701,447763.5,8.6,18.5,45.6,19.6,7.6,...,680,8760,3180,75,3180,6600,2215,3135,0,0
5,Gelderland (PV),"14,7","0,7",2133708,2141389.5,8.4,18.1,42.3,20.1,11.1,...,4025,47310,12125,460,15660,32735,13420,14775,5,0
6,Utrecht (PV),"15,0","0,7",1387643,1393850.0,7.6,15.0,36.1,22.2,19.0,...,2990,45775,9655,320,13930,23605,12425,9190,5,5
7,Noord-Holland (PV),"16,9","0,9",2952622,2964554.5,8.3,15.7,37.7,21.6,16.7,...,7660,100155,23885,510,28275,44710,36850,22005,5,10
8,Zuid-Holland (PV),"15,8","0,9",3804906,3822683.0,10.0,17.5,40.0,19.4,13.2,...,7170,92055,26160,845,28335,54905,25355,25275,5,110
9,Zeeland (PV),"15,8","1,0",391124,391379.0,8.8,20.0,47.1,17.6,6.6,...,890,6170,2020,170,2145,5240,1870,2865,5,0


In [6]:
merged_df.to_csv("data/save/merged_data.csv", index=False)